<a href="https://colab.research.google.com/github/MariaLon/homework/blob/hw04-sem2/hw04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача 1.
Посчитайте, как часто в выбранном вами тексте прилагательное стоит сразу перед существительным, а как часто - наоборот. Достаточно вывести количество вхождений (столько-то раз встретилось прил + сущ, столько-то раз - сущ + прил). Подумайте, насколько можно доверять этим цифрам.

Доверять цифрам нельзя, так как не учитывается граница предложения.

In [ ]:
import rnnmorph
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language ='ru')
f = open('text.txt',encoding='utf-8')
text = f.read()
import re
def tokenize(text):
  p = re.compile(r'(?m)\w+')
  return [o.group() for o in p.finditer(text) if o.group() != None]

POS = ' '.join([o.pos for o in predictor.predict(tokenize(text))])
print(POS)
p1 = re.compile(r'NOUN ADJ')
p2 = re.compile(r'ADJ NOUN')
print(f'NOUN + ADJ :{len(p1.findall(POS))}')
print(f'ADJ + NOUN :{len(p2.findall(POS))}')


1/1 [==============================] - 5s 5s/step
NOUN NOUN NOUN ADJ NOUN NOUN DET DET NOUN VERB ADJ NOUN CONJ DET NOUN ADJ CONJ PART ADJ NOUN ADP PRON PRON PART NOUN CONJ NOUN VERB ADP NOUN CONJ ADJ NOUN ADP ADJ NOUN NOUN VERB VERB ADJ NOUN NOUN ADJ NOUN NOUN NOUN ADV CONJ NOUN VERB ADP DET ADV ADJ ADJ NOUN CONJ ADV ADV PART VERB PRON ADP NOUN VERB ADJ NOUN ADP NOUN PRON NOUN NOUN ADJ NOUN ADP ADJ NOUN ADV VERB ADP NOUN NOUN VERB ADP NOUN ADJ NOUN NOUN ADJ NOUN ADV VERB ADJ PART DET PART NOUN ADJ ADV ADP DET NOUN ADP ADJ NOUN ADP NOUN NOUN VERB NOUN NOUN CONJ NOUN NOUN NOUN ADP NOUN ADP NOUN PART VERB ADJ NOUN NOUN DET VERB ADV ADV ADJ CONJ ADJ NOUN ADJ ADJ ADP ADJ NOUN CONJ ADJ ADJ NOUN VERB PART NUM VERB ADJ ADJ NOUN ADV ADJ CONJ CONJ DET PART ADP NOUN ADJ ADP ADJ NOUN NOUN VERB NOUN VERB VERB ADJ NOUN VERB DET ADJ NOUN ADP CONJ ADJ NOUN ADV ADJ ADJ NOUN ADV ADJ ADV VERB CONJ CONJ ADP NOUN DET NOUN CONJ NOUN VERB ADJ ADJ ADJ ADJ ADV ADV ADJ NOUN VERB ADP NUM NOUN VERB ADJ NOUN NOUN 

Задача 2 (3 балла).
Некоторые предлоги в русском языке могут управлять разными падежами (например, "я еду в Лондон" vs "я живу в Лондоне"). Давайте проанализируем эти предлоги и их падежи. Необходимо:

составить список таких предлогов (РГ-80 вам в помощь)
взять достаточно большой текст (можно большое художественное произведение)
сделать морфоразбор этого текста (лучше не pymorphy)
Посчитать, как часто и какие падежи встречаются у слова, идущего после предлога.
Примечания: во-первых, имейте в виду, что иногда после предлога могут идти самые неожиданные вещи: "я что, должен ехать на, черт побери, северный полюс?". Во-вторых, неплохо бы учитывать отсутствие пунктуации (конечно, в норме, как нам кажется, предлог обязательно требует зависимое, но! "да иди ты на!") Эти штуки можно отсеять, если просто учитывать только заранее определенные падежи, а не считать все, какие встретились (так и None можно огрести).

Если будете использовать RNNMoprph, возможно, понадобится регулярное выражение и немного терпения.

In [7]:
from nltk.tokenize import RegexpTokenizer
import rnnmorph
from rnnmorph.predictor import RNNMorphPredictor
import re

f = open('text.txt',encoding='utf-8')
text = f.read()
tokenizer = RegexpTokenizer(r'\w+|\$[\d\.]+|\S+')
tokens = tokenizer.tokenize(text)
predictor = RNNMorphPredictor(language ='ru')
M = predictor.predict(tokens)
D = dict()
p1 = re.compile(r'<[^>]+pos=ADP[^>]+><[^>]+Case=[^>]+>')
Res = [o.group() for o in p1.finditer(''.join([str(o) for o in M]))]
for r in Res:
  p2 = re.compile(r'((?<=\<normal_form=)[^;]+(?=.+<)|(?<=Case=)\w+\b)')
  pair = [o.group() for o in p2.finditer(r)]
  if pair[0] in D:
    if pair[1] in D[pair[0]]:
      D[pair[0]][pair[1]] += 1
    else:
      D[pair[0]][pair[1]] = 1
  else:
    D[pair[0]] = dict([(pair[1] , 1)])

for d in D:
  print(f'\n{d}')
  for c in D[d]:
    print(f'\t{c}={D[d][c]}')




1/1 [==============================] - 24s 24s/step

с
	Gen=58
	Ins=154

в
	Acc=261
	Loc=384
	Nom=2

со
	Ins=29
	Gen=8

на
	Acc=166
	Loc=169

у
	Gen=87

назад
	Acc=1
	Nom=1

вокруг
	Gen=4

после
	Gen=20

по
	Dat=86
	Acc=2

без
	Gen=16

кроме
	Gen=12

из
	Gen=98

за
	Ins=34
	Acc=38

про
	Acc=14

-за
	Gen=12

к
	Dat=139

ради
	Gen=1

до
	Gen=38

об
	Loc=11

от
	Gen=82

возле
	Gen=3

о
	Loc=69
	Acc=1

сквозь
	Acc=6

для
	Gen=8

близ
	Gen=2

над
	Ins=14

под
	Ins=33
	Acc=14

между
	Ins=16

перед
	Ins=17

-под
	Gen=3
	Acc=1

через
	Acc=28

мимо
	Gen=3
	Nom=1

пред
	Ins=1

при
	Loc=11

около
	Gen=13

вроде
	Gen=4

сзади
	Gen=1

впереди
	Gen=2

вследствие
	Gen=2

против
	Gen=1

вне
	Gen=1

среди
	Gen=5

поверх
	Gen=2

внутрь
	Gen=1

северо
	Dat=1

внизу
	Gen=1

насчёт
	Gen=1

вопреки
	Dat=1

меж
	Ins=1
	Gen=1

навстречу
	Dat=1

прежде
	Gen=1

вместо
	Gen=1
